In [1]:
import pandas as pd
import os
import lightgbm as lgb
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import time

from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor

import keras
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


In [2]:
price = pd.read_csv('./transpose-123 - transpose.csv')
path = 'pepper_supply/pepper'
o00 = pd.read_excel('./{}_00.xlsx'.format(path))
o01 = pd.read_excel('./{}_01.xlsx'.format(path))
o02 = pd.read_excel('./{}_02.xlsx'.format(path))
o03 = pd.read_excel('./{}_03.xlsx'.format(path))
o04 = pd.read_excel('./{}_04.xlsx'.format(path))
o05 = pd.read_excel('./{}_05.xlsx'.format(path))
o06 = pd.read_excel('./{}_06.xlsx'.format(path))
o07 = pd.read_excel('./{}_07.xlsx'.format(path))
o08 = pd.read_excel('./{}_08.xlsx'.format(path))
o09 = pd.read_excel('./{}_09.xlsx'.format(path))
o10 = pd.read_excel('./{}_10.xlsx'.format(path))
o11 = pd.read_excel('./{}_11.xlsx'.format(path))
o12 = pd.read_excel('./{}_12.xlsx'.format(path))
o13 = pd.read_excel('./{}_13.xlsx'.format(path))
o14 = pd.read_excel('./{}_14.xlsx'.format(path))
o15 = pd.read_excel('./{}_15.xlsx'.format(path))
o16 = pd.read_excel('./{}_16.xlsx'.format(path))
o17 = pd.read_excel('./{}_17.xlsx'.format(path))
o18 = pd.read_excel('./{}_18.xlsx'.format(path))
o19 = pd.read_excel('./{}_19.xlsx'.format(path))
# o20 = pd.read_excel('./{}_20.xlsx'.format(path))

supply = pd.concat([o00, o01, o02, o03, o04, o05, o06, o07, o08, o09, o10, o11, o12, o13, o14, o15, o16, o17, o18, o19])
supply['년도'] = supply['거래년월일'].apply(lambda x: str(x)[:4]).astype(str)
supply['월'] = supply['거래년월일'].apply(lambda x: str(x)[4:6]).astype(str)
supply['일'] = supply['거래년월일'].apply(lambda x: str(x)[6:8]).astype(str)
supply['거래량'] = supply['거래량'].apply(lambda x: ''.join(str(x).split(','))).astype(int)
sup_total = supply.groupby(['년도','월','일'], as_index=False).sum()
sup_total = sup_total[:-2]
sup_total.head()

,년도,월,일,시장코드,거래량,품목코드
0,2000,01,03,380201.0,130,120701.0
1,2000,01,06,380201.0,130,120701.0
2,2000,01,12,380201.0,130,120701.0
3,2000,01,14,380201.0,130,120701.0
4,2000,01,15,380201.0,130,120701.0


In [3]:
price['년도'] = price['년도'].apply(lambda x: str(x))
price['월'] = price['월/일'].apply(lambda x: str(x)[0:2]).astype(str)
price['일'] = price['월/일'].apply(lambda x: str(x)[3:5]).astype(str)
price.head()

,년도,월/일,평균,월,일
0,2000,01/03,"103,000",01,03
1,2000,01/04,"97,000",01,04
2,2000,01/05,"95,000",01,05
3,2000,01/06,"98,000",01,06
4,2000,01/07,"98,000",01,07


In [4]:
df = pd.merge(price, sup_total, on=['년도','월','일'])
df.head()

,년도,월/일,평균,월,일,시장코드,거래량,품목코드
0,2000,01/03,"103,000",01,03,380201.0,130,120701.0
1,2000,01/06,"98,000",01,06,380201.0,130,120701.0
2,2000,01/12,"97,000",01,12,380201.0,130,120701.0
3,2000,01/14,"95,000",01,14,380201.0,130,120701.0
4,2000,01/15,"91,000",01,15,380201.0,130,120701.0


In [5]:
# df2 = df[['평균']]
df2 = df[['년도','월','일','평균','거래량']]
df2['년도'] = df2['년도'].apply(lambda x: int(x))
df2['월'] = df2['월'].apply(lambda x: int(x))
df2['일'] = df2['일'].apply(lambda x: int(x))
df2['평균'] = df2['평균'].apply(lambda x: ''.join(str(x).split(','))).astype(int) 
df2.head()


/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

,년도,월,일,평균,거래량
0,2000,1,3,103000,130
1,2000,1,6,98000,130
2,2000,1,12,97000,130
3,2000,1,14,95000,130
4,2000,1,15,91000,130


In [6]:
# df2 = df[['평균']]
# df2['평균'] = df2['평균'].apply(lambda x: ''.join(str(x).split(','))).astype(int) 

a = list(df2['평균'])
df2['d+1'] = a
a.insert(0,0)
a = a[:-1]

df2['d-0'] = a
a.insert(0,0)
a = a[:-1]

df2['d-1'] = a
a.insert(0,0)
a = a[:-1]

df2['d-2'] = a
a.insert(0,0)
a = a[:-1]

df2['d-3'] = a
a.insert(0,0)
a = a[:-1]

df2['d-4'] = a
a.insert(0,0)
a = a[:-1]

df2['d-5'] = a
a.insert(0,0)
a = a[:-1]

df2['d-6'] = a
a.insert(0,0)
a = a[:-1]
del df2['평균']
df2 = df2[8:]
df2 = df2.reset_index(drop=True)

y = df2['d+1']
del df2['d+1']

df2

# from sklearn.preprocessing import MinMaxScaler
# mns = MinMaxScaler((0,1))
# mns.fit(df2)
# df4 = mns.transform(np.array(df2))

# y = df4[:,0]
# x = df4[:,1:]
# y

/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,년도,월,일,거래량,d-0,d-1,d-2,d-3,d-4,d-5,d-6
0,2000,1,31,130,95000,95000,95000,91000,95000,97000,98000
1,2000,2,8,489,95000,95000,95000,95000,91000,95000,97000
2,2000,2,12,130,90000,95000,95000,95000,95000,91000,95000
3,2000,2,14,130,64000,90000,95000,95000,95000,95000,91000
4,2000,2,19,720,57000,64000,90000,95000,95000,95000,95000
5,2000,2,28,4900,40000,57000,64000,90000,95000,95000,95000
6,2000,3,6,78,38000,40000,57000,64000,90000,95000,95000
7,2000,3,7,608,39400,38000,40000,57000,64000,90000,95000
8,2000,3,10,69,41400,39400,38000,40000,57000,64000,90000
9,2000,3,14,24,48800,41400,39400,38000,40000,57000,64000


In [7]:
seed_pred_result = []
rmse_result = []
for i in range(10):

    model = Sequential()

    model.add(Dense(64, input_dim=11))
    model.add(BatchNormalization())
    model.add(Activation('relu'))          
    model.add(Dense(4))
    model.add(Activation('relu'))
    # model.add(Dense(8))
    # model.add(BatchNormalization())
    # model.add(Activation('relu'))
    # model.add(Dense(4))
    # model.add(Activation('relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error',
                 optimizer='adam',
                 metrics=['accuracy'])

    rand_num = i
    x_train, x_test,y_train, y_test = train_test_split(df2, y, test_size=0.1, random_state=rand_num)

    gboost = GradientBoostingRegressor(random_state=rand_num)
    xgboost = xgb.XGBRegressor(random_state=rand_num)
    lightgbm = lgb.LGBMRegressor(random_state=rand_num)
    lasso = Lasso(alpha=1.0, random_state=rand_num)

    models = [{'model':xgboost, 'name':'XGBoost'},
              {'model':gboost, 'name': 'GradientBoosting'},
              {'model':lightgbm, 'name':'LightGBM'},
              {'model':lasso, 'name': 'Lasso'}]
    result = []
    rmse = []
    result.append(y_test)

    for m in models:
        m['model'].fit(x_train.values, y_train)
        pickle.dump(m['model'], open("pima_{}_{}.pickle.dat".format(i, m['name']), "wb"))

    for m in models:
        loaded_model = pickle.load(open("pima_{}_{}.pickle.dat".format(i, m['name']), "rb"))
        predictions = loaded_model.predict(x_test.values) 
        result.append(predictions)
        rmse.append({m['name']:sqrt(mean_squared_error(y_test, predictions))})
    
    # 모델 저장 폴더 설정
    MODEL_DIR = './model_{}/'.format(i)
    if not os.path.exists(MODEL_DIR):
        os.mkdir(MODEL_DIR)
        
    # 모델 저장 조건 설정
    modelpath = MODEL_DIR + 'keras_{epoch:02d}-{val_loss:.4f}.hdf5'
    checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=350)
    def scheduler(epoch):
        if epoch > 80:
            return 0.006
        if epoch < 170:
            return 0.005
        if epoch < 260:
            return 0.004
        if epoch < 350:
            return 0.003
        if epoch > 480:
            return 0.002
        if epoch < 570:
            return 0.001
        if epoch > 680:
            return 0.006
        if epoch < 770:
            return 0.005
        if epoch < 860:
            return 0.004
        if epoch < 950:
            return 0.003
        if epoch > 1080:
            return 0.002
        if epoch < 1170:
            return 0.001
        else:
            return 0.007

    lr_schedule = keras.callbacks.LearningRateScheduler(scheduler)

    history = model.fit(x_train, y_train, validation_split=0.1, epochs=1500, batch_size=1024, callbacks=[checkpointer, early_stop, lr_schedule])
    
#     mns.inverse_transform(model.predict())
    modelPred = model.predict(x_test)
    Y_prediction = modelPred.flatten()

    result.append(Y_prediction)
    rmse.append({'keras':sqrt(mean_squared_error(y_test, modelPred))})

    dict = {'y': result[0], 'xgboost': result[1], 'gboost': result[2], 'lightgbm':result[3], 'lasso':result[4],'keras': result[5]}

    df_res = pd.DataFrame(dict)
    df_res['average'] = df_res[['xgboost', 'gboost', 'lightgbm', 'lasso', 'keras']].apply(lambda x: round(x.mean()), axis=1)
    rmse.append({'average':sqrt(mean_squared_error(y_test, df_res['average']))})

    rmse_result.append(rmse)
    seed_pred_result.append(df_res)    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/home/florence/.local/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 41212573934.44804, tolerance: 434684637.44574577
  positive)


Train on 3402 samples, validate on 378 samples
Epoch 1/1500
3402/3402 [==============================] - 1s 222us/step - loss: 4649359949.6578 - acc: 0.0000e+00 - val_loss: 4499608064.0000 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 4499608064.00000, saving model to ./model_0/keras_01-4499608064.0000.hdf5
Epoch 2/1500
3402/3402 [==============================] - 0s 3us/step - loss: 4649355942.1517 - acc: 0.0000e+00 - val_loss: 4499606528.0000 - val_acc: 0.0000e+00

Epoch 00002: val_loss improved from 4499608064.00000 to 4499606528.00000, saving model to ./model_0/keras_02-4499606528.0000.hdf5
Epoch 3/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4649348609.8060 - acc: 0.0000e+00 - val_loss: 4499598848.0000 - val_acc: 0.0000e+00

Epoch 00003: val_loss improved from 4499606528.00000 to 4499598848.00000, saving model to ./model_0/keras_03-4499598848.0000.hdf5
Epoch 4/1500
3402/3402 [==============================] - 0s 3us/step - loss: 4649334

Epoch 29/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4639129458.2293 - acc: 0.0000e+00 - val_loss: 4484787200.0000 - val_acc: 0.0000e+00

Epoch 00029: val_loss improved from 4485908480.00000 to 4484787200.00000, saving model to ./model_0/keras_29-4484787200.0000.hdf5
Epoch 30/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4637860606.9465 - acc: 0.0000e+00 - val_loss: 4483475968.0000 - val_acc: 0.0000e+00

Epoch 00030: val_loss improved from 4484787200.00000 to 4483475968.00000, saving model to ./model_0/keras_30-4483475968.0000.hdf5
Epoch 31/1500
3402/3402 [==============================] - 0s 3us/step - loss: 4636483444.9383 - acc: 0.0000e+00 - val_loss: 4481578496.0000 - val_acc: 0.0000e+00

Epoch 00031: val_loss improved from 4483475968.00000 to 4481578496.00000, saving model to ./model_0/keras_31-4481578496.0000.hdf5
Epoch 32/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4635016319.9248 - acc: 0.0000e+00 - val

3402/3402 [==============================] - 0s 4us/step - loss: 4544387430.4903 - acc: 0.0000e+00 - val_loss: 4373321728.0000 - val_acc: 0.0000e+00

Epoch 00057: val_loss improved from 4392686080.00000 to 4373321728.00000, saving model to ./model_0/keras_57-4373321728.0000.hdf5
Epoch 58/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4538043775.4733 - acc: 0.0000e+00 - val_loss: 4378550784.0000 - val_acc: 0.0000e+00

Epoch 00058: val_loss did not improve from 4373321728.00000
Epoch 59/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4531469111.8354 - acc: 0.0000e+00 - val_loss: 4371536384.0000 - val_acc: 0.0000e+00

Epoch 00059: val_loss improved from 4373321728.00000 to 4371536384.00000, saving model to ./model_0/keras_59-4371536384.0000.hdf5
Epoch 60/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4524607419.6731 - acc: 0.0000e+00 - val_loss: 4357890048.0000 - val_acc: 0.0000e+00

Epoch 00060: val_loss improved from 43

...

3402/3402 [==============================] - 0s 4us/step - loss: 4664680724.3175 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00182: val_loss did not improve from 4728944128.00000
Epoch 183/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680359.8072 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00183: val_loss did not improve from 4728944128.00000
Epoch 184/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680557.5638 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00184: val_loss did not improve from 4728944128.00000
Epoch 185/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680633.1146 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00185: val_loss did not improve from 4728944128.00000
Epoch 186/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680510.3069 - acc: 0.0000e+00 -

3402/3402 [==============================] - 0s 4us/step - loss: 4664680629.2016 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00218: val_loss did not improve from 4728944128.00000
Epoch 219/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680324.5902 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00219: val_loss did not improve from 4728944128.00000
Epoch 220/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680621.9777 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00220: val_loss did not improve from 4728944128.00000
Epoch 221/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680509.4039 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00221: val_loss did not improve from 4728944128.00000
Epoch 222/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680325.6437 - acc: 0.0000e+00 -

3402/3402 [==============================] - 0s 4us/step - loss: 4664680415.1911 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00254: val_loss did not improve from 4728944128.00000
Epoch 255/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680384.7901 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00255: val_loss did not improve from 4728944128.00000
Epoch 256/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680338.1352 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00256: val_loss did not improve from 4728944128.00000
Epoch 257/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680823.6473 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00257: val_loss did not improve from 4728944128.00000
Epoch 258/1500
3402/3402 [==============================] - 0s 5us/step - loss: 4664680777.5944 - acc: 0.0000e+00 -

3402/3402 [==============================] - 0s 4us/step - loss: 4664680552.4468 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00290: val_loss did not improve from 4728944128.00000
Epoch 291/1500
3402/3402 [==============================] - 0s 3us/step - loss: 4664680676.4586 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00291: val_loss did not improve from 4728944128.00000
Epoch 292/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680515.7249 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00292: val_loss did not improve from 4728944128.00000
Epoch 293/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680578.0317 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00293: val_loss did not improve from 4728944128.00000
Epoch 294/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680603.0147 - acc: 0.0000e+00 -

3402/3402 [==============================] - 0s 4us/step - loss: 4664680533.1828 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00326: val_loss did not improve from 4728944128.00000
Epoch 327/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680504.2869 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00327: val_loss did not improve from 4728944128.00000
Epoch 328/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680636.1246 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00328: val_loss did not improve from 4728944128.00000
Epoch 329/1500
3402/3402 [==============================] - 0s 4us/step - loss: 4664680652.9806 - acc: 0.0000e+00 - val_loss: 4728944128.0000 - val_acc: 0.0000e+00

Epoch 00329: val_loss did not improve from 4728944128.00000
Epoch 330/1500
3402/3402 [==============================] - 0s 5us/step - loss: 4664680508.5009 - acc: 0.0000e+00 -

In [8]:
rmse_result[:4]

[[{'XGBoost': 4657.130103201285},
  {'GradientBoosting': 4168.8032955255385},
  {'LightGBM': 4303.670886087159},
  {'Lasso': 4263.586777981639},
  {'keras': 4333.5152467105345},
  {'average': 4114.0456243276985}],
 [{'XGBoost': 5034.449214487736},
  {'GradientBoosting': 4625.934710458606},
  {'LightGBM': 4617.797193480077},
  {'Lasso': 4438.929685221648},
  {'keras': 4480.478625186342},
  {'average': 4428.6606917402605}],
 [{'XGBoost': 4628.0824017760015},
  {'GradientBoosting': 4630.121470929163},
  {'LightGBM': 4552.513909705088},
  {'Lasso': 4619.197939468868},
  {'keras': 4615.722816093922},
  {'average': 4411.35046084088}],
 [{'XGBoost': 5082.848876730722},
  {'GradientBoosting': 4628.000320603883},
  {'LightGBM': 4659.10666141021},
  {'Lasso': 4427.995219281457},
  {'keras': 5376.5017855522},
  {'average': 4466.9256398790685}]]

In [9]:
seed_pred_result[0].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
3037,78000,77826.125000,79402.854849,80449.709866,79452.805294,78680.109375,79162.0
3485,116000,105800.882812,108938.910279,112018.455552,105549.325573,104720.796875,107406.0
3347,78000,76217.765625,75561.771605,75138.598947,75949.657438,75175.828125,75609.0
72,34400,33877.492188,33811.652081,34312.981919,34320.287580,34618.265625,34188.0
2284,39000,41137.832031,40148.349904,39454.159405,40128.397586,40015.343750,40177.0


In [10]:
seed_pred_result[1].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
3604,51400,54367.156250,55170.619237,53599.142932,53627.176116,53864.558594,54126.0
781,53200,56213.566406,56886.906950,56633.845933,56929.541769,57858.679688,56905.0
2165,161000,155272.906250,156661.530831,153000.451880,153848.532998,154895.015625,154736.0
1217,101600,99488.289062,102213.888317,102542.253241,103182.896168,104071.210938,102300.0
1006,72400,70328.226562,69291.108745,70954.899578,69037.551243,70165.640625,69955.0


In [11]:
seed_pred_result[2].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
1296,37800,35799.695312,34613.869331,34976.128092,34148.998201,34576.648438,34823.0
140,74600,71278.867188,70637.437420,71077.535798,71889.827302,71642.851562,71305.0
2560,74400,73788.109375,75557.301549,75172.878367,74649.585667,74396.296875,74713.0
674,22800,22681.369141,23106.630841,22789.298711,22910.041530,23552.267578,23008.0
118,57000,53275.773438,56380.296204,55842.010256,55162.598005,54795.265625,55091.0


In [12]:
seed_pred_result[8].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
2669,41000,43562.101562,43528.044963,44479.674064,44006.394003,44242.792969,43964.0
1197,80800,55103.515625,55175.189388,58238.547295,54271.554407,54526.957031,55463.0
1133,32600,31558.628906,31202.279415,31450.567685,31765.257194,32356.423828,31667.0
1121,22000,20235.919922,20583.048237,20541.201388,20513.186332,21305.035156,20636.0
740,36800,38532.105469,38285.756303,38666.800084,37885.330440,38531.203125,38380.0


In [13]:
# seed_pred_result[4].head(5)

In [14]:
df2 = df[['평균']]
df2['평균'] = df2['평균'].apply(lambda x: ''.join(str(x).split(','))).astype(int) 



a = list(df2['평균'])
df2['d+1'] = a
a.insert(0,0)
a = a[:-1]

df2['d-0'] = a
a.insert(0,0)
a = a[:-1]

df2['d-1'] = a
a.insert(0,0)
a = a[:-1]

df2['d-2'] = a
a.insert(0,0)
a = a[:-1]

df2['d-3'] = a
a.insert(0,0)
a = a[:-1]

df2['d-4'] = a
a.insert(0,0)
a = a[:-1]

df2['d-5'] = a
a.insert(0,0)
a = a[:-1]

df2['d-6'] = a
a.insert(0,0)
a = a[:-1]
del df2['평균']
df2 = df2[8:]
df2 = df2.reset_index(drop=True)

from sklearn.preprocessing import MinMaxScaler
mns = MinMaxScaler((0,1))
mns.fit(df2)
df4 = mns.transform(np.array(df2))

y = df4[:,0]
x = df4[:,1:]

/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pand

In [15]:
model = Sequential()
model.add(keras.layers.Reshape((1,7), input_shape=(7,)))
model.add(LSTM(32,dropout=0.3,recurrent_activation='relu',recurrent_initializer='glorot_uniform'))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='relu'))
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
model.summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1, 7)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                5120      
_________________________________________________________________
dense_31 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
___________________________

In [16]:
# 모델 저장 폴더 설정
MODEL_DIR = './lstm/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

    
x_train, x_test,y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=rand_num)

# 모델 저장 조건 설정
modelpath = MODEL_DIR + 'keras_{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=350)
def scheduler(epoch):
    if epoch > 80:
        return 0.006
    if epoch < 170:
        return 0.005
    if epoch < 260:
        return 0.004
    if epoch < 350:
        return 0.003
    if epoch > 480:
        return 0.002
    if epoch < 570:
        return 0.001
    if epoch > 680:
        return 0.006
    if epoch < 770:
        return 0.005
    if epoch < 860:
        return 0.004
    if epoch < 950:
        return 0.003
    if epoch > 1080:
        return 0.002
    if epoch < 1170:
        return 0.001
    else:
        return 0.007

lr_schedule = keras.callbacks.LearningRateScheduler(scheduler)

history = model.fit(x_train, y_train, validation_split=0.1, epochs=1500, batch_size=1024, callbacks=[checkpointer, early_stop, lr_schedule])


modelPred = model.predict(x_test)
Y_prediction = modelPred.flatten()

print(sqrt(mean_squared_error(y_test, modelPred)))
print(y_test[:3])
print(Y_prediction[:3])


Train on 3402 samples, validate on 378 samples
Epoch 1/1500
3402/3402 [==============================] - 1s 348us/step - loss: 0.0628 - acc: 2.9394e-04 - val_loss: 0.0321 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.03212, saving model to ./lstm/keras_01-0.0321.hdf5
Epoch 2/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0292 - acc: 2.9394e-04 - val_loss: 0.0287 - val_acc: 0.0000e+00

Epoch 00002: val_loss improved from 0.03212 to 0.02869, saving model to ./lstm/keras_02-0.0287.hdf5
Epoch 3/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0308 - acc: 2.9394e-04 - val_loss: 0.0166 - val_acc: 0.0000e+00

Epoch 00003: val_loss improved from 0.02869 to 0.01661, saving model to ./lstm/keras_03-0.0166.hdf5
Epoch 4/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0170 - acc: 2.9394e-04 - val_loss: 0.0106 - val_acc: 0.0000e+00

Epoch 00004: val_loss improved from 0.01661 to 0.01056, saving model to ./l


Epoch 00038: val_loss did not improve from 0.00375
Epoch 39/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0043 - acc: 5.8789e-04 - val_loss: 0.0075 - val_acc: 0.0000e+00

Epoch 00039: val_loss did not improve from 0.00375
Epoch 40/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0046 - acc: 5.8789e-04 - val_loss: 0.0093 - val_acc: 0.0000e+00

Epoch 00040: val_loss did not improve from 0.00375
Epoch 41/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0040 - acc: 5.8789e-04 - val_loss: 0.0064 - val_acc: 0.0000e+00

Epoch 00041: val_loss did not improve from 0.00375
Epoch 42/1500
3402/3402 [==============================] - ETA: 0s - loss: 0.0041 - acc: 9.7656e-0 - 0s 7us/step - loss: 0.0043 - acc: 5.8789e-04 - val_loss: 0.0093 - val_acc: 0.0000e+00

Epoch 00042: val_loss did not improve from 0.00375
Epoch 43/1500
3402/3402 [==============================] - 0s 5us/step - loss: 0.0043 - acc: 5.8789e-04 - val_loss: 0.

3402/3402 [==============================] - 0s 7us/step - loss: 0.0031 - acc: 5.8789e-04 - val_loss: 0.0108 - val_acc: 0.0000e+00

Epoch 00080: val_loss did not improve from 0.00375
Epoch 81/1500
3402/3402 [==============================] - 0s 7us/step - loss: 0.0029 - acc: 5.8789e-04 - val_loss: 0.0122 - val_acc: 0.0000e+00

Epoch 00081: val_loss did not improve from 0.00375
Epoch 82/1500
3402/3402 [==============================] - 0s 7us/step - loss: 0.0030 - acc: 5.8789e-04 - val_loss: 0.0142 - val_acc: 0.0000e+00

Epoch 00082: val_loss did not improve from 0.00375
Epoch 83/1500
3402/3402 [==============================] - 0s 7us/step - loss: 0.0031 - acc: 5.8789e-04 - val_loss: 0.0119 - val_acc: 0.0000e+00

Epoch 00083: val_loss did not improve from 0.00375
Epoch 84/1500
3402/3402 [==============================] - 0s 7us/step - loss: 0.0029 - acc: 5.8789e-04 - val_loss: 0.0126 - val_acc: 0.0000e+00

Epoch 00084: val_loss did not improve from 0.00375
Epoch 85/1500
3402/3402 [====


Epoch 00121: val_loss did not improve from 0.00375
Epoch 122/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0028 - acc: 2.9394e-04 - val_loss: 0.0140 - val_acc: 0.0000e+00

Epoch 00122: val_loss did not improve from 0.00375
Epoch 123/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0026 - acc: 5.8789e-04 - val_loss: 0.0168 - val_acc: 0.0000e+00

Epoch 00123: val_loss did not improve from 0.00375
Epoch 124/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0029 - acc: 5.8789e-04 - val_loss: 0.0139 - val_acc: 0.0000e+00

Epoch 00124: val_loss did not improve from 0.00375
Epoch 125/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0026 - acc: 5.8789e-04 - val_loss: 0.0159 - val_acc: 0.0000e+00

Epoch 00125: val_loss did not improve from 0.00375
Epoch 126/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0027 - acc: 5.8789e-04 - val_loss: 0.0143 - val_acc: 0.0000e+00

Epoch 001

3402/3402 [==============================] - 0s 6us/step - loss: 0.0024 - acc: 5.8789e-04 - val_loss: 0.0161 - val_acc: 0.0000e+00

Epoch 00163: val_loss did not improve from 0.00375
Epoch 164/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0025 - acc: 5.8789e-04 - val_loss: 0.0170 - val_acc: 0.0000e+00

Epoch 00164: val_loss did not improve from 0.00375
Epoch 165/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0024 - acc: 5.8789e-04 - val_loss: 0.0152 - val_acc: 0.0000e+00

Epoch 00165: val_loss did not improve from 0.00375
Epoch 166/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0026 - acc: 5.8789e-04 - val_loss: 0.0149 - val_acc: 0.0000e+00

Epoch 00166: val_loss did not improve from 0.00375
Epoch 167/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0022 - acc: 5.8789e-04 - val_loss: 0.0162 - val_acc: 0.0000e+00

Epoch 00167: val_loss did not improve from 0.00375
Epoch 168/1500
3402/3402 

3402/3402 [==============================] - 0s 5us/step - loss: 0.0024 - acc: 5.8789e-04 - val_loss: 0.0173 - val_acc: 0.0000e+00

Epoch 00204: val_loss did not improve from 0.00375
Epoch 205/1500
3402/3402 [==============================] - 0s 5us/step - loss: 0.0023 - acc: 5.8789e-04 - val_loss: 0.0156 - val_acc: 0.0000e+00

Epoch 00205: val_loss did not improve from 0.00375
Epoch 206/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0023 - acc: 5.8789e-04 - val_loss: 0.0160 - val_acc: 0.0000e+00

Epoch 00206: val_loss did not improve from 0.00375
Epoch 207/1500
3402/3402 [==============================] - 0s 5us/step - loss: 0.0026 - acc: 5.8789e-04 - val_loss: 0.0162 - val_acc: 0.0000e+00

Epoch 00207: val_loss did not improve from 0.00375
Epoch 208/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0022 - acc: 5.8789e-04 - val_loss: 0.0167 - val_acc: 0.0000e+00

Epoch 00208: val_loss did not improve from 0.00375
Epoch 209/1500
3402/3402 

3402/3402 [==============================] - 0s 6us/step - loss: 0.0024 - acc: 5.8789e-04 - val_loss: 0.0189 - val_acc: 0.0000e+00

Epoch 00245: val_loss did not improve from 0.00375
Epoch 246/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0023 - acc: 5.8789e-04 - val_loss: 0.0167 - val_acc: 0.0000e+00

Epoch 00246: val_loss did not improve from 0.00375
Epoch 247/1500
3402/3402 [==============================] - 0s 5us/step - loss: 0.0024 - acc: 5.8789e-04 - val_loss: 0.0157 - val_acc: 0.0000e+00

Epoch 00247: val_loss did not improve from 0.00375
Epoch 248/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0022 - acc: 5.8789e-04 - val_loss: 0.0185 - val_acc: 0.0000e+00

Epoch 00248: val_loss did not improve from 0.00375
Epoch 249/1500
3402/3402 [==============================] - 0s 5us/step - loss: 0.0023 - acc: 5.8789e-04 - val_loss: 0.0181 - val_acc: 0.0000e+00

Epoch 00249: val_loss did not improve from 0.00375
Epoch 250/1500
3402/3402 

3402/3402 [==============================] - 0s 7us/step - loss: 0.0021 - acc: 5.8789e-04 - val_loss: 0.0179 - val_acc: 0.0000e+00

Epoch 00286: val_loss did not improve from 0.00375
Epoch 287/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0022 - acc: 5.8789e-04 - val_loss: 0.0180 - val_acc: 0.0000e+00

Epoch 00287: val_loss did not improve from 0.00375
Epoch 288/1500
3402/3402 [==============================] - 0s 7us/step - loss: 0.0023 - acc: 5.8789e-04 - val_loss: 0.0151 - val_acc: 0.0000e+00

Epoch 00288: val_loss did not improve from 0.00375
Epoch 289/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0024 - acc: 5.8789e-04 - val_loss: 0.0151 - val_acc: 0.0000e+00

Epoch 00289: val_loss did not improve from 0.00375
Epoch 290/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0023 - acc: 5.8789e-04 - val_loss: 0.0193 - val_acc: 0.0000e+00

Epoch 00290: val_loss did not improve from 0.00375
Epoch 291/1500
3402/3402 

3402/3402 [==============================] - 0s 5us/step - loss: 0.0019 - acc: 5.8789e-04 - val_loss: 0.0175 - val_acc: 0.0000e+00

Epoch 00327: val_loss did not improve from 0.00375
Epoch 328/1500
3402/3402 [==============================] - 0s 5us/step - loss: 0.0022 - acc: 5.8789e-04 - val_loss: 0.0180 - val_acc: 0.0000e+00

Epoch 00328: val_loss did not improve from 0.00375
Epoch 329/1500
3402/3402 [==============================] - 0s 5us/step - loss: 0.0021 - acc: 2.9394e-04 - val_loss: 0.0152 - val_acc: 0.0000e+00

Epoch 00329: val_loss did not improve from 0.00375
Epoch 330/1500
3402/3402 [==============================] - 0s 5us/step - loss: 0.0022 - acc: 5.8789e-04 - val_loss: 0.0183 - val_acc: 0.0000e+00

Epoch 00330: val_loss did not improve from 0.00375
Epoch 331/1500
3402/3402 [==============================] - 0s 6us/step - loss: 0.0022 - acc: 5.8789e-04 - val_loss: 0.0167 - val_acc: 0.0000e+00

Epoch 00331: val_loss did not improve from 0.00375
Epoch 332/1500
3402/3402 

3402/3402 [==============================] - 0s 5us/step - loss: 0.0022 - acc: 5.8789e-04 - val_loss: 0.0152 - val_acc: 0.0000e+00

Epoch 00368: val_loss did not improve from 0.00375
0.11570105986632456
[0.03508772 0.04775828 0.26315789]
[0.07743155 0.07751559 0.15654999]


In [17]:
# x_train, x_test,y_train, y_test = train_test_split(df2, y, test_size=0.3, random_state=rand_num)
# x_train

In [18]:
# 다른 주소에서 다른 화면을 보여주는것을 라우팅이라 함